## ARCH Model Estimation

**Functions**

`arch.arch_model`

### Exercise 44

Download 10 years of data for the S&P 500 and the EUR/USD exchange rate from FRED.


In [1]:
import pandas as pd
import pandas_datareader as pdr
import os

if not os.path.exists("./data/arch-data.h5"):
    sp500 = pdr.get_data_fred("SP500", start="2010-1-1", end="2020-1-1").dropna()
    eurusd = pdr.get_data_fred("DEXUSEU", start="2010-1-1", end="2020-1-1").dropna()
    
    with pd.HDFStore("./data/arch-data.h5", mode="w") as store:
        store.put("sp500", sp500)
        store.put("eurusd", eurusd)

sp500 = pd.read_hdf("./data/arch-data.h5", "sp500")        
eurusd = pd.read_hdf("./data/arch-data.h5", "eurusd")


### Exercise 45

Estimate a GARCH(1,1) and a GJR-GARCH(1,1,1) to the returns of both series.

**Note**: You need to install arch using

```bash
pip install arch
```

which contains ARCH and related models.

In [2]:
from arch import arch_model

sp500_ret = 100 * sp500.pct_change().dropna()
eurusd_ret = 100 * eurusd.pct_change().dropna()

In [3]:
garch = arch_model(sp500_ret)
res_sp500 = garch.fit()
res_sp500.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 3014.5904796994937
Iteration:      2,   Func. Count:     16,   Neg. LLF: 3012.5320163673546
Iteration:      3,   Func. Count:     25,   Neg. LLF: 3010.801852261159
Iteration:      4,   Func. Count:     34,   Neg. LLF: 3008.514323297097
Iteration:      5,   Func. Count:     41,   Neg. LLF: 2998.34678892632
Iteration:      6,   Func. Count:     48,   Neg. LLF: 2998.1375359845188
Iteration:      7,   Func. Count:     54,   Neg. LLF: 2997.091706804308
Iteration:      8,   Func. Count:     60,   Neg. LLF: 2997.0652133388126
Iteration:      9,   Func. Count:     66,   Neg. LLF: 2997.0613460443938
Iteration:     10,   Func. Count:     72,   Neg. LLF: 2997.060737138707
Iteration:     11,   Func. Count:     78,   Neg. LLF: 2997.060725515726
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2997.060725516014
            Iterations: 11
            Function evaluations: 78
            Gradient evaluation

C:\Anaconda\envs\anaconda-core\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.8663. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                  SP500   R-squared:                      -0.002
Mean Model:             Constant Mean   Adj. R-squared:                 -0.002
Vol Model:                      GARCH   Log-Likelihood:               -2997.06
Distribution:                  Normal   AIC:                           6002.12
Method:            Maximum Likelihood   BIC:                           6025.43
                                        No. Observations:                 2510
Date:                Fri, Jan 10 2020   Df Residuals:                     2506
Time:                        17:44:25   Df Model:                            4
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0817  1.397e-02      5.845  5.081e-09 [5.429e-02,  0.109]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0360  8.098e-03      4.452  8.514e-06 [2.018e-02,5.192e-02]
alpha[1]       0.1688  2.472e-02      6.830  8.479e-12     [  0.120,  0.217]
beta[1]        0.7923  2.409e-02     32.887 3.400e-237     [  0.745,  0.839]
============================================================================

Covariance estimator: robust
"""

In [4]:
gjr_garch = arch_model(sp500_ret, p=1, o=1, q=1)
gjr_res_sp500 = gjr_garch.fit()
gjr_res_sp500.summary()


Iteration:      1,   Func. Count:      7,   Neg. LLF: 2945.418133462882
Iteration:      2,   Func. Count:     18,   Neg. LLF: 2944.3417483263506
Iteration:      3,   Func. Count:     29,   Neg. LLF: 2943.7187965445237
Iteration:      4,   Func. Count:     38,   Neg. LLF: 2936.582909117798
Iteration:      5,   Func. Count:     47,   Neg. LLF: 2931.8549330769347
Iteration:      6,   Func. Count:     55,   Neg. LLF: 2931.082208305511
Iteration:      7,   Func. Count:     62,   Neg. LLF: 2929.7171136854913
Iteration:      8,   Func. Count:     70,   Neg. LLF: 2929.705424848964
Iteration:      9,   Func. Count:     77,   Neg. LLF: 2929.7034836293406
Iteration:     10,   Func. Count:     84,   Neg. LLF: 2929.703200371813
Iteration:     11,   Func. Count:     91,   Neg. LLF: 2929.703198450817
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2929.7031984509217
            Iterations: 11
            Function evaluations: 91
            Gradient evaluati

C:\Anaconda\envs\anaconda-core\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.8663. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                   Constant Mean - GJR-GARCH Model Results                    
==============================================================================
Dep. Variable:                  SP500   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -2929.70
Distribution:                  Normal   AIC:                           5869.41
Method:            Maximum Likelihood   BIC:                           5898.55
                                        No. Observations:                 2510
Date:                Fri, Jan 10 2020   Df Residuals:                     2505
Time:                        17:44:25   Df Model:                            5
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0417  1.353e-02      3.078  2.086e-03 [1.513e-02,6.818e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0349  5.870e-03      5.942  2.808e-09  [2.338e-02,4.639e-02]
alpha[1]   1.5545e-11  3.042e-02  5.110e-10      1.000 [-5.962e-02,5.962e-02]
gamma[1]       0.2799  4.204e-02      6.659  2.764e-11      [  0.198,  0.362]
beta[1]        0.8176  2.372e-02     34.469 2.309e-260      [  0.771,  0.864]
=============================================================================

Covariance estimator: robust
"""

In [5]:
garch = arch_model(eurusd_ret)
res_eurusd = garch.fit()
res_eurusd.summary()

C:\Anaconda\envs\anaconda-core\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.3051. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1909.670911265314
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1909.624029248601
Iteration:      3,   Func. Count:     25,   Neg. LLF: 1909.5927218523254
Iteration:      4,   Func. Count:     32,   Neg. LLF: 1899.5314975665528
Iteration:      5,   Func. Count:     38,   Neg. LLF: 1890.8762765427095
Iteration:      6,   Func. Count:     45,   Neg. LLF: 1887.7773933380681


Iteration:      7,   Func. Count:     55,   Neg. LLF: 1886.4594708340137
Iteration:      8,   Func. Count:     64,   Neg. LLF: 1886.4590654686854
Iteration:      9,   Func. Count:     71,   Neg. LLF: 1886.4329318801788
Iteration:     10,   Func. Count:     77,   Neg. LLF: 1886.4203363174297
Iteration:     11,   Func. Count:     83,   Neg. LLF: 1886.4203040415516
Iteration:     12,   Func. Count:     89,   Neg. LLF: 1886.4203000155449
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1886.4203000140574
            Iterations: 12
            Function evaluations: 89
            Gradient evaluations: 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                DEXUSEU   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:               -1886.42
Distribution:                  Normal   AIC:                           3780.84
Method:            Maximum Likelihood   BIC:                           3804.14
                                        No. Observations:                 2501
Date:                Fri, Jan 10 2020   Df Residuals:                     2497
Time:                        17:44:25   Df Model:                            4
                                  Mean Model                                  
==============================================================================
                  coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------------------------------
mu         -7.6712e-03  9.141e-03     -0.839      0.401 [-2.559e-02,1.024e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      6.3182e-04  6.328e-04      0.998      0.318 [-6.085e-04,1.872e-03]
alpha[1]       0.0272  6.165e-03      4.413  1.020e-05  [1.512e-02,3.929e-02]
beta[1]        0.9705  6.731e-03    144.185      0.000      [  0.957,  0.984]
=============================================================================

Covariance estimator: robust
"""

In [6]:
gjr_garch = arch_model(eurusd_ret, p=1, o=1, q=1)
gjr_res_eurusd = gjr_garch.fit()
gjr_res_eurusd.summary()



C:\Anaconda\envs\anaconda-core\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.3051. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)


Iteration:      1,   Func. Count:      7,   Neg. LLF: 1908.7131549283613
Iteration:      2,   Func. Count:     18,   Neg. LLF: 1908.7017243622008
Iteration:      3,   Func. Count:     28,   Neg. LLF: 1908.6773347942121
Iteration:      4,   Func. Count:     35,   Neg. LLF: 1889.4413702783654
Iteration:      5,   Func. Count:     44,   Neg. LLF: 1882.525466006632
Iteration:      6,   Func. Count:     57,   Neg. LLF: 1881.7040754139696


Iteration:      7,   Func. Count:     65,   Neg. LLF: 1880.8465398551466
Iteration:      8,   Func. Count:     74,   Neg. LLF: 1880.6114932116336
Iteration:      9,   Func. Count:     81,   Neg. LLF: 1879.5646991817252
Iteration:     10,   Func. Count:     89,   Neg. LLF: 1879.4200019887553
Iteration:     11,   Func. Count:     99,   Neg. LLF: 1879.0677367934366
Iteration:     12,   Func. Count:    107,   Neg. LLF: 1878.7974042427315
Iteration:     13,   Func. Count:    114,   Neg. LLF: 1878.6558613780458
Iteration:     14,   Func. Count:    121,   Neg. LLF: 1878.5979598271808
Iteration:     15,   Func. Count:    128,   Neg. LLF: 1878.5926945357276
Iteration:     16,   Func. Count:    135,   Neg. LLF: 1878.5917752110588
Iteration:     17,   Func. Count:    142,   Neg. LLF: 1878.5917323749304
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1878.5917323806534
            Iterations: 17
            Function evaluations: 142
            Gradient 

<class 'statsmodels.iolib.summary.Summary'>
"""
                   Constant Mean - GJR-GARCH Model Results                    
==============================================================================
Dep. Variable:                DEXUSEU   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -1878.59
Distribution:                  Normal   AIC:                           3767.18
Method:            Maximum Likelihood   BIC:                           3796.31
                                        No. Observations:                 2501
Date:                Fri, Jan 10 2020   Df Residuals:                     2496
Time:                        17:44:25   Df Model:                            5
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu            -0.0122  9.172e-03     -1.331      0.183 [-3.018e-02,5.770e-03]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      8.8896e-05  3.517e-04      0.253      0.800 [-6.004e-04,7.782e-04]
alpha[1]   6.1209e-03  6.774e-03      0.904      0.366 [-7.156e-03,1.940e-02]
gamma[1]       0.0232  5.937e-03      3.907  9.338e-05  [1.156e-02,3.483e-02]
beta[1]        0.9817  6.105e-03    160.801      0.000      [  0.970,  0.994]
=============================================================================

Covariance estimator: robust
"""

### Exercise 46
Exercise 66. Comment on the asymmetry.

  * Compare robust and non-robust standard errors.
  * Plot the fit variance and fit volatility.
  * Plot the standardized residuals.


In [7]:
gjr_garch = arch_model(sp500_ret, p=1, o=1, q=1)
gjr_res_sp500_non = gjr_garch.fit(cov_type='classic')
gjr_res_sp500_non.summary()


C:\Anaconda\envs\anaconda-core\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.8663. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)


Iteration:      1,   Func. Count:      7,   Neg. LLF: 2945.418133462882
Iteration:      2,   Func. Count:     18,   Neg. LLF: 2944.3417483263506
Iteration:      3,   Func. Count:     29,   Neg. LLF: 2943.7187965445237
Iteration:      4,   Func. Count:     38,   Neg. LLF: 2936.582909117798


Iteration:      5,   Func. Count:     47,   Neg. LLF: 2931.8549330769347
Iteration:      6,   Func. Count:     55,   Neg. LLF: 2931.082208305511
Iteration:      7,   Func. Count:     62,   Neg. LLF: 2929.7171136854913
Iteration:      8,   Func. Count:     70,   Neg. LLF: 2929.705424848964
Iteration:      9,   Func. Count:     77,   Neg. LLF: 2929.7034836293406
Iteration:     10,   Func. Count:     84,   Neg. LLF: 2929.703200371813
Iteration:     11,   Func. Count:     91,   Neg. LLF: 2929.703198450817


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2929.7031984509217
            Iterations: 11
            Function evaluations: 91
            Gradient evaluations: 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                   Constant Mean - GJR-GARCH Model Results                    
==============================================================================
Dep. Variable:                  SP500   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -2929.70
Distribution:                  Normal   AIC:                           5869.41
Method:            Maximum Likelihood   BIC:                           5898.55
                                        No. Observations:                 2510
Date:                Fri, Jan 10 2020   Df Residuals:                     2505
Time:                        17:44:25   Df Model:                            5
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0417  1.358e-02      3.067  2.163e-03 [1.503e-02,6.828e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0349  4.262e-03      8.184  2.749e-16  [2.653e-02,4.324e-02]
alpha[1]   1.5545e-11  1.750e-02  8.882e-10      1.000 [-3.430e-02,3.430e-02]
gamma[1]       0.2799  3.051e-02      9.174  4.541e-20      [  0.220,  0.340]
beta[1]        0.8176  1.794e-02     45.568      0.000      [  0.782,  0.853]
=============================================================================

Covariance estimator: classic
"""

In [8]:
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(16,9))

fig = gjr_res_sp500.plot(annualize='D')

C:\Anaconda\envs\anaconda-core\lib\site-packages\pandas\plotting\_matplotlib\converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [9]:
plt.rc("figure", figsize=(16,6))
vol = gjr_res_sp500.conditional_volatility
var = vol**2
ax = var.plot()

In [10]:
ax = gjr_res_sp500.std_resid.plot()